In [1]:
!pip install keras
!pip install tensorflow
!pip install pydot

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 133kB 646kB/s ta 0:00:01
  Running setup.py bdist_wheel for pydot ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/6a/a5/14/25541ebcdeaf97a37b6d05c7ff15f5bd20f5e91b99d313e5b4
Successfully built pydot
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
np.random.seed(13)
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape
from IPython.display import SVG
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot, plot_model
import gensim

Using TensorFlow backend.


In [3]:
path = get_file('alice.txt', origin="http://www.gutenberg.org/cache/epub/11/pg11.txt")
corpus = open(path).readlines()[0:200]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters="'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2

In [4]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            s = index-window_size
            e = index+window_size+1
                    
            in_words = []
            labels = []
            for i in range(s, e):
                if i != index and 0 <= i < L:
                    in_words.append([word] )
                    labels.append(words[i])

            x = np.array(in_words,dtype=np.int32)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [5]:
skipgram = Sequential()
skipgram.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, units=V, activation='softmax'))
# SVG(model_to_dot(skipgram, show_shapes=True).create(prog='dot', format='svg'))

In [6]:
skipgram.layers

In [7]:
skipgram.compile(loss='categorical_crossentropy', optimizer="adadelta")

In [8]:
for ite in range(10):
    loss = 0.
    for i, (x, y) in enumerate(generate_data(corpus, window_size, V)):
        if i<4:
            print(' ', i, x.shape, y.shape)
        loss += skipgram.train_on_batch(x, y)

    print(ite, loss)

  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
0 11489.388918876648
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
1 10761.77791762352
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
2 10547.217913866043
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
3 10571.68703341484
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
4 10600.938851118088
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
5 10587.000823497772
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
6 10563.15277004242
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
7 10537.503711938858
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
8 10507.644553780556
  0 (2, 1) (2, 755)
  1 (3, 1) (3, 755)
  2 (4, 1) (4, 755)
  3 (4, 1) (4, 755)
9 10476.657511

In [9]:
f = open('vectors-sg.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))
vectors = skipgram.get_weights()[0]
for num, (word, i) in enumerate(tokenizer.word_index.items()):
#     if num> 38:
#         continue
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word.strip(), str_vec))
f.close()

In [10]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors-sg.txt', binary=False)

In [11]:
w2v.most_similar(positive=['alice'])

[('she', 0.47049450874328613),
 ('what', 0.4238196313381195),
 ('got', 0.40675997734069824),
 ('had', 0.4031747281551361),
 ('about', 0.3843131363391876),
 ('no', 0.3503741919994354),
 ('not', 0.3490760028362274),
 ('poor', 0.3486499786376953),
 ('how', 0.34808725118637085),
 ('right', 0.3421412706375122)]